In [1]:
import os

In [2]:
%pwd

'/Users/dhruv/PROJECTS/Kaggle_Projects/Rohlik-Orders-Forecasting-Challenge/notebook'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/dhruv/PROJECTS/Kaggle_Projects/Rohlik-Orders-Forecasting-Challenge'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_type: str
    kaggle_competition_name: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from rohlik_forecasting.constants import *
from rohlik_forecasting.utils.main_utils import read_yaml,create_directories

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,) :
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config['artifacts_root']])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config['data_ingestion']
        create_directories([config['root_dir']])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config['root_dir'],
            source_type = config['source_type'],
            kaggle_competition_name = config['kaggle_competition_name'],
            local_data_file = config['local_data_file'],
            unzip_dir = config['unzip_dir']
        )
        return data_ingestion_config  

In [20]:
import os
import zipfile
import kaggle
from rohlik_forecasting import logger 
#from rohlik_forecasting.utils.main_utils import get_size
from kaggle.api.kaggle_api_extended import KaggleApi
import shutil

In [27]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self) -> str:
        # Authenticate with Kaggle API
        api = KaggleApi()
        api.authenticate()
 
        try: 
            dataset_url = f"{self.config.kaggle_competition_name}"
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.logging.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            api.competition_download_files(dataset_url, path=zip_download_dir)
            logger.logging.info(f"Downloaded dataset from kaggle to {zip_download_dir}")
            #logger.info(f"Downloaded dataset from kaggle to {self.config.root_dir}")
        
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        zip_file_path = os.path.join(self.config.local_data_file,'rohlik-orders-forecasting-challenge.zip')
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.logging.info(f"Extracted dataset to {unzip_path}")

In [29]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e